**Install python dependencies**

In [ ]:
!pip install -q -r ./dependencies/requirements.txt

**Load python libraries**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree

**Load Data**

In [31]:
X = pd.read_csv('./data/inputs.csv')
y = pd.read_csv('./data/label.csv')

**Drop Bettings Odds and Position**

In [32]:
X = X.drop(['odds_win', 'odds_draw', 'odds_lose', 'position_normalized', 'matchday'], axis='columns')

**Split data into test and split**

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

**Descriptive Analytics**

In [35]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366781 entries, 348646 to 121958
Data columns (total 4 columns):
 #   Column                 Non-Null Count   Dtype
---  ------                 --------------   -----
 0   is_home                366781 non-null  bool 
 1   weighted_occurences    366781 non-null  int64
 2   team_name_normalized   366781 non-null  int64
 3   event_type_normalized  366781 non-null  int64
dtypes: bool(1), int64(3)
memory usage: 11.5 MB


**Create Deicision Tree**

In [ ]:
model = tree.DecisionTreeRegressor()
model = model.fit(inputs, label)

model.score(inputs, label)

- GridSearchCV